<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 16</h1>
</div>

# 1. Strings: `String` and `&str`
# 2. Lifetimes

# <font color="red">1. Strings: `String` and `&str`</font>
# 2. Lifetimes

## Rust and strings

* We have avoided this topic so far


* It's complicated


* Unicode is complicated


* Advantages: internationalization and emojis out of the box

* **Rust:** Unicode strings are a first–class citizen


* **Classical programming languages:**
  * ASCII strings are the default
  * Easier to manage
  * Additional libraries needed to deal with Unicode


## Reminder: Single characters (Unicode scalar values)

* Type: `char`
* Size: 4 bytes
* Note the single quotes!

In [2]:
let a : char = 'a';
let b = '🦕';

Dinosaurs:<br>
&nbsp;&nbsp;&nbsp;🦕 (U+1F995)<br>
&nbsp;&nbsp;&nbsp;🦖 (U+1F996)

In [78]:
// Mayan numeral (not all unicode characters are supported everywhere)
let c = '𝋥';

In [79]:
std::mem::size_of_val(&a)

4

In [80]:
std::mem::size_of_val(&b)

4

In [81]:
let c = char::from_u32(0x1F995);
println!("UTF32 character {:?}", c);
let c = char::from_u32(0x890710e7);
println!("UTF32 character {:?}", c);


UTF32 character Some('🦕')
UTF32 character None


## String literals

* String literal${}={}$when you create a string `"like this"`
* Note the double quotes
* What type are they?

In [82]:
let sample = "Hello, DS210!";

In [4]:
let sample: String = "Hello, DS210!".to_string();

In [84]:
let sample: &str = "Hello, DS210!";

`&str` is a **string slice**, internally behaves like `&[u8]`

In [85]:
let byte_escape = "I'm writing \x52\x75\x73\x74!";
println!("What are you doing\x3F (\\x3F means ?) {}", byte_escape);

// This will not work as hex can only be 2 bytes long (two characters)
// let unicode_codepoint = "\x211D";

// ...or Unicode code points.
let unicode_codepoint = "\u{211D}";
let character_name = "\"DOUBLE-STRUCK CAPITAL R\"";
println!("Unicode character {} (U+211D) is called {}",
                unicode_codepoint, character_name );

What are you doing? (\x3F means ?) I'm writing Rust!
Unicode character ℝ (U+211D) is called "DOUBLE-STRUCK CAPITAL R"


## Encoding of characters

`a` and `🦕` were both 4 bytes

In [7]:
std::mem::size_of_val("a")

1

In [87]:
std::mem::size_of_val("🦕")

4

Characters need 1–4 bytes to be encoded.

In [88]:
let dinos = "🦕🦖";
std::mem::size_of_val(dinos)

8

In [89]:
let mixed = "a🦖b🦕";
std::mem::size_of_val(mixed)

10

In [43]:
// Iterating through characters
for (i, c) in mixed.chars().enumerate() {
    println!("{} {} {}", i, c, std::mem::size_of_val(&c));
}


0 a 4
1 🦖 4
2 b 4
3 🦕 4


()

Can select substrings, but they must be aligned with actual characters (or runtime error)

In [44]:
dinos[0..1]

thread '<unnamed>' panicked at 'byte index 1 is not a char boundary; it is inside '🦕' (bytes 0..4) of `🦕🦖`', src/lib.rs:216:40
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::str::slice_error_fail_rt
   3: core::str::slice_error_fail
   4: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   5: _run_user_code_27
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [45]:
let dinos = "🦕🦖";
dinos[0..4]

"🦕"

In [46]:
let sample = "Hello, world!";
sample[7..]

"world!"

## Strings

* String type is dynamic: `Vec<u8>` internally
* Can add characters and strings to the end

In [90]:
let mut sample = String::new();

//append string
sample.push_str("abc");
sample

"abc"

In [91]:
// append character
sample.push('d');
sample

"abcd"

## Converting literals to type `String`

Use `.to_string()` or `String::from(...)`

In [92]:
let string_1 = "This is a test".to_string();
let string_2 = String::from("This is a test");
string_1 == string_2

true

Can also use macro `format!(...)`:
  * same syntax as `println!(...)`
  * produces an object of type `String`

In [93]:
let sample: String = format!("{} == {}",string_1,string_2);
sample

"This is a test == This is a test"

## String concatenation via `+`

* Takes ownership of the first parameter
* Second parameter: `&str`

In [94]:
let string_1 = "abc".to_string();
let string_2 = "def".to_string();

In [95]:
string_1 + &string_2

"abcdef"

Why `+` takes ownership of `string_1`:
 * reason: efficiency
 * no need to copy the content of the first string (unless the container size has to be increased)

## Writing generic code
* Use string slices &str if possible
* This will work with `String` and `&str`

In [96]:
fn show(message: &str) {
    println!("{}",message);
}

In [54]:
// automatic conversion to &str from &String
let mut my_string = String::from("ds210");
show(&my_string);
show("ds210");

ds210
ds210


In [55]:
// &str has the same limitation as any slice (not extensible)
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{} {} {} {}", s1, s2, s3, s4);

String1 String2 str1 str2


In [56]:
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{}", s1+s2);

Error: mismatched types

In [57]:
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{}", s1+s3);








String1str1


In [58]:
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{}", s1+&s2);

String1String2


In [59]:
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{}", s3+s1);

Error: cannot add `String` to `&str`

In [60]:
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{}", *s3+s4);

Error: cannot add `&str` to `str`

In [61]:
let s1 = "String1".to_string();
let s2 = "String2".to_string();
let s3 = "str1";
let s4 = "str2";
println!("{}", s3+s4);

Error: cannot add `&str` to `&str`

# 1. Strings: `String` and `&str`
# <font color="red">2. Lifetimes</font>



## Lifetimes

* How long your reference is valid
* Important when sharing references
  * Example: via function output

In [97]:
// This is fine
fn ref_to_max_simple(x:&mut i32, y:&mut i32) -> i32 {
    if *x >= *y {
        *x
    } else {
        *y
    }
}

**Challenge:** return the reference to the greater of two integers 

In [98]:
// But this is not
fn ref_to_max(x:&mut i32, y:&mut i32) -> &mut i32 {
    if *x >= *y {
        x
    } else {
        y
    }
}

Error: missing lifetime specifier

## Why does this matter?
### Imagine this piece of code
```
let z: &i32;  // A variable
let x: i32 = 100;
{
let y: i32 = 55
z = ref_to_max(&x, &y);
}
println!("{}", z);
```

## Specifying lifetimes

<code>'t</code> specifies how long a reference lives (`t` is some string) 

 * immutable example: <code>&'t i32</code>
 * mutable example: <code>&'t mut i32</code>

In [99]:
fn ref_to_max<'a>(x:&'a mut i32, y:&'a mut i32) -> &'a mut i32 {
    if *x >= *y {
        x
    } else {
        y
    }
}

In [66]:
{
let mut z: &mut i32;
let mut x = 13;
{
    let mut y = 9;
    println!("{} {}",x,y);
    z = ref_to_max(&mut x, &mut y); //1
    println!("{} {}",x,y);
    *ref_to_max(&mut x, &mut y) = 1; //2
    println!("{} {}",x,y);
    *ref_to_max(&mut x, &mut y) = 0; //3
    println!("{} {}",x,y);
}
};

13 9
13 9
1 9
1 0


In [9]:
fn ref_to_max_immut<'a>(x:&'a i32, y:&'a i32) -> &'a i32 {
    if *x >= *y {
        x
    } else {
        y
    }
}

{
let z: &i32;  // A variable
let mut x: i32 = 100;
{
    let mut y: i32 = 55;
    z = ref_to_max_immut(&x, &y);
    //*z = 22;
    println!("{}", *z);
    println!("{} {}",x,y);
}
println!("{}", z);
}

Error: `y` does not live long enough

## Applying this function

* Different references may have different lifetimes
* Rust will automatically select the shortest 

In [100]:
{
let mut x = 1;
let z: &mut i32;
let ref1: &mut i32;
{
    let mut y = 10;
    ref1 = &mut y;
    {
        let ref2 = &mut x;
        z = ref_to_max(ref1, ref2);
    }
    *ref1 *= -1;
}
*z = 3;
// (x,y)
}

Error: `y` does not live long enough

Error: cannot use `*ref1` because it was mutably borrowed

## Multiple lifetimes possible

In [101]:
fn multiple<'a, 'b>(x:&'a str, y:&'b str) -> (&'a str,&'b str) {
    (x,y)
}
multiple("abc","def")

("abc", "def")

## String literals are forever

* Memory for them assigned in the code

* Their references do not expire

* Can be specified by `'static`

In [102]:
let example: &'static str = "abc";

# The 3 rules of lifetimes

### * The compiler assigns a lifetime parameter to each parameter that’s a reference. A function with one parameter gets one lifetime parameter: fn foo<'a>(x: &'a i32); a function with two parameters gets two separate lifetime parameters: fn foo<'a, 'b>(x: &'a i32, y: &'b i32); and so on.

### * If there is exactly one input lifetime parameter, that lifetime is assigned to all output lifetime parameters: fn foo<'a>(x: &'a i32) -> &'a i32.

### * If there are multiple input lifetime parameters, but one of them is &self or &mut self because this is a method, the lifetime of self is assigned to all output lifetime parameters. 

All functions `get_shorter` below equivalent

In [12]:
struct TwoStrings{
    a: String,
    b: String,
}

In [13]:
fn get_shorter_1(ts:&TwoStrings) -> &str {
    if ts.a.len() < ts.b.len() {
        &ts.a
    } else {
        &ts.b
    }
}

In [14]:
fn get_shorter_2<'a>(ts:&'a TwoStrings) -> &'a str {
    if ts.a.len() < ts.b.len() {
        &ts.a
    } else {
        &ts.b
    }
}

In [15]:
fn get_shorter_3<'a>(ts:&'a TwoStrings) -> &str {
    if ts.a.len() < ts.b.len() {
        &ts.a
    } else {
        &ts.b
    }
}

In [16]:
let two_strings: TwoStrings = TwoStrings {a:"abc".to_string(), b:"defg".to_string()};
println!("1 {}", get_shorter_1(&two_strings));
println!("2 {}", get_shorter_2(&two_strings));
println!("3 {}", get_shorter_3(&two_strings));


1 abc
2 abc
3 abc


# What about the 3rd rule
**Example 2:** one of the lifetime parameters is `&self` or `&mut self` => its lifetime used as the lifetime of output

Methods `get_longer` below equivalent

In [17]:
impl TwoStrings {
    fn get_longer_1(&self, _unused:&TwoStrings) -> &str {
        if self.a.len() < self.b.len() {
            &self.a
        } else {
            &self.b
        }
    }

    fn get_longer_2<'a, 'b>(&'a self, _unused:&'b TwoStrings) -> &'a str {
        if self.a.len() < self.b.len() {
            &self.a
        } else {
            &self.b
        }
    }
}

In [18]:
impl TwoStrings {
    fn get_longer_3(&self, unused:&TwoStrings) -> &str {
        if self.a.len() < self.b.len() {
            &unused.a
        } else {
            &unused.b
        }
    }
    
}

Error: lifetime may not live long enough

In [19]:
impl TwoStrings {
    fn get_longer_3<'a, 'b>(&'a self, unused:&'b TwoStrings) -> &'b str {
        if self.a.len() < self.b.len() {
            &unused.a
        } else {
            &unused.b
        }
    }
    
}

In [20]:
let other_strings: TwoStrings = TwoStrings {a:"foobar".to_string(), b:"barfoo".to_string()};

println!("1 {} ", two_strings.get_longer_1(&other_strings));
println!("2 {} ", two_strings.get_longer_2(&other_strings));
println!("3 {} ", two_strings.get_longer_3(&other_strings));

1 abc 
2 abc 
3 foobar 




# <font color="red">Closures (anonymous functions)</font>

## Closures (anonymous functions)

We have seen them before in Python (as lambda functions):

```Python
lambda a b: a * b
```

In Rust (with implicit or explicit type specification):
```
|a, b| a * b
|a: i32, b: i32| -> i32 {a * b}
```

In [39]:
{
  let f = |a, b| a * b;
  let x = 10;
  let y = 20;
  println!("{}",f(x,y));
};

200


### Digression -- You can assign regular functions to variable as well

In [40]:
{
    fn median2(arr: &mut [i32]) -> i32 {
        arr.sort();
        println!("{}", arr[2]);
        arr[2]
    }

let f = median2;
f(&mut [1,4,5,6,4]);
}

4


()

## Sample application: lazy evaluation of a value
Compute a value only if needed

In [21]:
// What does it compute?
fn expensive_function(i:u32) -> u128 {
    if i <= 1 {
        i as u128
    } else {
        expensive_function(i-1) + expensive_function(i-2)
    }
}

In [22]:
expensive_function(44)

701408733

In [23]:
// This function always computes expensive_function(44), even if not needed.
// Method unwrap_or takes a default value as a parameter.
fn value_or_fib44(input:Option<u128>) -> u128 {
    input.unwrap_or(expensive_function(44))
}

In [24]:
use std::time::SystemTime;
let d = SystemTime::now();
// slow
let ret = value_or_fib44(None);
let elapsed = d.elapsed().unwrap().as_millis();
println!("{} {}",ret, elapsed);

701408733 4973


In [25]:
let d = SystemTime::now();
// slow
let ret = value_or_fib44(Some(123));
let elapsed = d.elapsed().unwrap().as_millis();
println!("{} {}", ret, elapsed);

123 4974


## Sample application: lazy evaluation of a value
Compute a value only if needed

In [26]:
// This function computes expensive_function(44) only if needed.
// Method unwrap_or_else's parameter is a function that computes
// the default value, not the default value itself. 
fn value_or_fib44_version_2(input:Option<u128>) -> u128 {
    input.unwrap_or_else(|| expensive_function(44))
}

In [27]:
// slow
let d = SystemTime::now();
value_or_fib44_version_2(None);
let elapsed = d.elapsed().unwrap().as_millis();
println!("{}", elapsed);

4981


In [28]:
// fast
let d = SystemTime::now();
value_or_fib44_version_2(Some(1));
let elapsed = d.elapsed().unwrap().as_millis();
println!("{}", elapsed);

0


* This programing pattern appears in many places.
* Another example: default value for an entry in HashMap

In [29]:
let mut map = std::collections::HashMap::<i32,i32>::new();
map.insert(1, 1);
*map.entry(1).or_insert_with(|| expensive_function(44) as i32) *= -1;
*map.entry(2).or_insert_with(|| expensive_function(44) as i32) *= -1;
println!("{}:{:?}    {}:{:?}",1,map.get(&1),2,map.get(&2));

1:Some(-1)    2:Some(-701408733)


## Read section 8.2 for strings, section 10.3 for lifetimes and section 13.1 and 13.2 for closures and iterators